###  importing the required libraries and loading the data:

In [1]:
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/home/suyog/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/suyog/.local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from subprocess import check_output
print(check_output(["ls", "/home/suyog/github/Machine_Learning_with_Python/data"]).decode("utf8")) 

1data_description.txt
1sample_submission.csv
1test.csv
1train.csv
av_hackathon_test.csv
av_hackathon_train.csv
av_hackathon_train_modified.csv



In [3]:
#norma read_csv with any extra params thrrew utf-8 error
train = pd.read_csv('/home/suyog/github/Machine_Learning_with_Python/data/av_hackathon_train.csv', engine='python')
#train = pd.read_csv('/home/suyog/github/Machine_Learning_with_Python/data/av_hackathon_train.csv', engine='latin1')

In [4]:
train.columns

Index(['ID', 'Gender', 'City', 'Monthly_Income', 'DOB', 'Lead_Creation_Date',
       'Loan_Amount_Applied', 'Loan_Tenure_Applied', 'Existing_EMI',
       'Employer_Name', 'Salary_Account', 'Mobile_Verified', 'Var5', 'Var1',
       'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'Interest_Rate',
       'Processing_Fee', 'EMI_Loan_Submitted', 'Filled_Form', 'Device_Type',
       'Var2', 'Source', 'Var4', 'LoggedIn', 'Disbursed'],
      dtype='object')

In [5]:
train.head()

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,...,NaN,NaN,NaN,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,...,NaN,NaN,NaN,N,Web-browser,B,S134,3,1,0


In [6]:
target = 'Disbursed'
IDcol = 'ID'


## Feature Engineering


In [7]:
## get available methods
[method_name for method_name in dir(train) if callable(getattr(train, method_name))]

/home/suyog/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


['__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__div__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclass

In [8]:
train.dtypes

ID                        object
Gender                    object
City                      object
Monthly_Income             int64
DOB                       object
Lead_Creation_Date        object
Loan_Amount_Applied      float64
Loan_Tenure_Applied      float64
Existing_EMI             float64
Employer_Name             object
Salary_Account            object
Mobile_Verified           object
Var5                       int64
Var1                      object
Loan_Amount_Submitted    float64
Loan_Tenure_Submitted    float64
Interest_Rate            float64
Processing_Fee           float64
EMI_Loan_Submitted       float64
Filled_Form               object
Device_Type               object
Var2                      object
Source                    object
Var4                       int64
LoggedIn                   int64
Disbursed                  int64
dtype: object

In [9]:
train.describe()

,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Var5,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Var4,LoggedIn,Disbursed
count,8.702000e+04,8.694900e+04,86949.000000,8.694900e+04,87020.000000,5.240700e+04,52407.000000,27726.000000,27420.000000,27726.000000,87020.000000,87020.000000,87020.000000
mean,5.884997e+04,2.302507e+05,2.131399,3.696228e+03,4.961503,3.950106e+05,3.891369,19.197474,5131.150839,10999.528377,2.949805,0.029350,0.014629
std,2.177511e+06,3.542068e+05,2.014193,3.981021e+04,5.670385,3.082481e+05,1.165359,5.834213,4725.837644,7512.323050,1.697720,0.168785,0.120062
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,5.000000e+04,1.000000,11.990000,200.000000,1176.410000,0.000000,0.000000,0.000000
25%,1.650000e+04,0.000000e+00,0.000000,0.000000e+00,0.000000,2.000000e+05,3.000000,15.250000,2000.000000,6491.600000,1.000000,0.000000,0.000000
50%,2.500000e+04,1.000000e+05,2.000000,0.000000e+00,2.000000,3.000000e+05,4.000000,18.000000,4000.000000,9392.970000,3.000000,0.000000,0.000000
75%,4.000000e+04,3.000000e+05,4.000000,3.500000e+03,11.000000,5.000000e+05,5.000000,20.000000,6250.000000,12919.040000,5.000000,0.000000,0.000000
max,4.445544e+08,1.000000e+07,10.000000,1.000000e+07,18.000000,3.000000e+06,6.000000,37.000000,50000.000000,144748.280000,7.000000,1.000000,1.000000


In [10]:
train.describe(include = 'all')

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
count,87020,87020,86017,8.702000e+04,87020,87020,8.694900e+04,86949.000000,8.694900e+04,86949,...,27726.000000,27420.000000,27726.000000,87020,87020,87020,87020,87020.000000,87020.000000,87020.000000
unique,87020,2,697,NaN,11345,92,NaN,NaN,NaN,43567,...,NaN,NaN,NaN,2,2,7,30,NaN,NaN,NaN
top,ID072471J10,Male,Delhi,NaN,11-Nov-80,03-Jul-15,NaN,NaN,NaN,0,...,NaN,NaN,NaN,N,Web-browser,B,S122,NaN,NaN,NaN
freq,1,49848,12527,NaN,306,2315,NaN,NaN,NaN,4914,...,NaN,NaN,NaN,67530,64316,37280,38567,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.884997e+04,NaN,NaN,2.302507e+05,2.131399,3.696228e+03,NaN,...,19.197474,5131.150839,10999.528377,NaN,NaN,NaN,NaN,2.949805,0.029350,0.014629
std,NaN,NaN,NaN,2.177511e+06,NaN,NaN,3.542068e+05,2.014193,3.981021e+04,NaN,...,5.834213,4725.837644,7512.323050,NaN,NaN,NaN,NaN,1.697720,0.168785,0.120062
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,...,11.990000,200.000000,1176.410000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,1.650000e+04,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,...,15.250000,2000.000000,6491.600000,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000
50%,NaN,NaN,NaN,2.500000e+04,NaN,NaN,1.000000e+05,2.000000,0.000000e+00,NaN,...,18.000000,4000.000000,9392.970000,NaN,NaN,NaN,NaN,3.000000,0.000000,0.000000
75%,NaN,NaN,NaN,4.000000e+04,NaN,NaN,3.000000e+05,4.000000,3.500000e+03,NaN,...,20.000000,6250.000000,12919.040000,NaN,NaN,NaN,NaN,5.000000,0.000000,0.000000


In [11]:
#[train[c].describe() for c in train.columns]

In [6]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

Number of variables,26
Number of observations,87020
Total Missing (%),8.9%
Total size in memory,17.3 MiB
Average record size in memory,208.0 B
Numeric,10
Categorical,12
Boolean,2
Date,0
Text (Unique),1
Rejected,1


### Dropping ID variable which is not meaningful

In [13]:
train.drop(['ID'], axis=1, inplace = True)

### 1.  'City' variable dropped because of too many categories

In [14]:
train['City'].describe()

count     86017
unique      697
top       Delhi
freq      12527
Name: City, dtype: object

In [15]:
train['City'].astype(str).describe()

count     87020
unique      698
top       Delhi
freq      12527
Name: City, dtype: object

In [16]:
train.drop(['City'], axis=1, inplace = True)

### 2. DOB converted to Age and DOB dropped

In [17]:
train['DOB'].head()

0    23-May-78
1    07-Oct-85
2    10-Oct-81
3    30-Nov-87
4    17-Feb-84
Name: DOB, dtype: object

In [18]:
from datetime import datetime as dt

In [19]:
now = pd.Timestamp(dt.now())
now

Timestamp('2019-01-01 04:19:27.944222')

In [4]:
train['DOB'] = pd.to_datetime(train['DOB'], format='%d-%b-%y')

0   1978-05-23
1   1985-10-07
2   1981-10-10
3   1987-11-30
4   1984-02-17
Name: DOB, dtype: datetime64[ns]

In [21]:
train['DOB'] = train['DOB'].where(train['DOB'] < now, train['DOB'] -  np.timedelta64(100, 'Y')) 
train['age'] = (now - train['DOB']).astype('<m8[Y]') 

In [22]:
train['age'].head()

0    40.0
1    33.0
2    37.0
3    31.0
4    34.0
Name: age, dtype: float64

In [23]:
train.drop(['DOB'], axis=1, inplace = True)

### 3. EMI_Loan_Submitted_Missing created which is 1 if EMI_Loan_Submitted was missing else 0, and Original variable EMI_Loan_Submitted dropped

In [24]:
train['EMI_Loan_Submitted'].head(6)

0        NaN
1    6762.90
2        NaN
3        NaN
4        NaN
5    6978.92
Name: EMI_Loan_Submitted, dtype: float64

In [25]:
train['EMI_Loan_Submitted'].isna().sum()

59294

In [26]:
train.loc[train.EMI_Loan_Submitted.notna(), 'EMI_Loan_Submitted_Missing'] = 0
train.loc[train.EMI_Loan_Submitted.isna(), 'EMI_Loan_Submitted_Missing'] = 1
train['EMI_Loan_Submitted_Missing'].head(6)

0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
5    0.0
Name: EMI_Loan_Submitted_Missing, dtype: float64

In [27]:
train.drop(['EMI_Loan_Submitted'], axis=1, inplace = True)

### 4. EmployerName dropped because of too many categories

In [28]:
train['Employer_Name'].describe()

count     86949
unique    43567
top           0
freq       4914
Name: Employer_Name, dtype: object

In [29]:
train.drop(['Employer_Name'], axis=1, inplace = True)

### 5. Existing_EMI imputed with 0 (median) since only 111 values were missing

In [30]:
train['Existing_EMI'].head(10)

0        0.0
1        0.0
2        0.0
3        0.0
4    25000.0
5    15000.0
6        0.0
7     2597.0
8        0.0
9        0.0
Name: Existing_EMI, dtype: float64

In [31]:
train['Existing_EMI'].describe()

count    8.694900e+04
mean     3.696228e+03
std      3.981021e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      3.500000e+03
max      1.000000e+07
Name: Existing_EMI, dtype: float64

In [32]:
train['Existing_EMI'].isna().sum()

71

In [33]:
existing_emi_array = train[train['Existing_EMI']!=np.nan]['Existing_EMI']
train['Existing_EMI'].replace(np.nan, existing_emi_array.median(), inplace = True)
#train['Existing_EMI'].fillna(train['Existing_EMI'].median())

In [34]:
train['Existing_EMI'].isna().sum()

0

### 6. Interest_Rate_Missing created which is 1 if Interest_Rate was missing else 0 | Original variable Interest_Rate dropped

In [35]:
train['Interest_Rate'].head()

0      NaN
1    13.25
2      NaN
3      NaN
4      NaN
Name: Interest_Rate, dtype: float64

In [36]:
train['Interest_Rate'].describe()

count    27726.000000
mean        19.197474
std          5.834213
min         11.990000
25%         15.250000
50%         18.000000
75%         20.000000
max         37.000000
Name: Interest_Rate, dtype: float64

In [37]:
train.loc[train.Interest_Rate.notna(), 'Interest_Rate_Missing'] = 0
train.loc[train.Interest_Rate.isna(), 'Interest_Rate_Missing'] = 1
train['Interest_Rate_Missing'].head()

0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
Name: Interest_Rate_Missing, dtype: float64

In [38]:
train.drop(['Interest_Rate'], axis=1, inplace = True)

### 7. Lead_Creation_Date dropped because made little intuitive impact on outcome

In [39]:
train['Lead_Creation_Date'].describe()

count         87020
unique           92
top       03-Jul-15
freq           2315
Name: Lead_Creation_Date, dtype: object

In [40]:
train.drop(['Lead_Creation_Date'], axis=1, inplace = True)

### 8. Loan_Amount_Applied, Loan_Tenure_Applied imputed with median values

In [41]:
train['Loan_Amount_Applied'].isna().sum()

71

In [42]:
filtered_array = train[train['Loan_Amount_Applied']!=np.nan]['Loan_Amount_Applied']
train['Loan_Amount_Applied'].replace(np.nan, filtered_array.median(), inplace = True)

In [43]:
train['Loan_Amount_Applied'].isna().sum()

0

In [44]:
train['Loan_Tenure_Applied'].isna().sum()

71

In [45]:
filtered_array = train[train['Loan_Tenure_Applied']!=np.nan]['Loan_Amount_Applied']
train['Loan_Tenure_Applied'].replace(np.nan, filtered_array.median(), inplace = True)

In [46]:
train['Loan_Tenure_Applied'].isna().sum()

0

### 9. Loan_Amount_Submitted_Missing created which is 1 if Loan_Amount_Submitted was missing else 0 | Original variable Loan_Amount_Submitted dropped

In [47]:
train['Loan_Amount_Submitted'].head()

0         NaN
1    200000.0
2    450000.0
3    920000.0
4    500000.0
Name: Loan_Amount_Submitted, dtype: float64

In [48]:
train.loc[train.Loan_Amount_Submitted.notna(), 'Loan_Amount_Submitted_Missing'] = 0
train.loc[train.Loan_Amount_Submitted.isna(), 'Loan_Amount_Submitted_Missing'] = 1
train.drop(['Loan_Amount_Submitted'], axis=1, inplace = True)

In [49]:
train['Loan_Amount_Submitted_Missing'].head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Loan_Amount_Submitted_Missing, dtype: float64

### 10. Loan_Tenure_Submitted_Missing created which is 1 if Loan_Tenure_Submitted was missing else 0 | Original variable Loan_Tenure_Submitted droppe

In [50]:
train['Loan_Tenure_Submitted'].head()

0    NaN
1    2.0
2    4.0
3    5.0
4    2.0
Name: Loan_Tenure_Submitted, dtype: float64

In [51]:
train.loc[train.Loan_Tenure_Submitted.notna(), 'Loan_Tenure_Submitted_Missing'] = 0
train.loc[train.Loan_Tenure_Submitted.isna(), 'Loan_Tenure_Submitted_Missing'] = 1
train.drop(['Loan_Tenure_Submitted'], axis=1, inplace = True)

In [52]:
train['Loan_Tenure_Submitted_Missing'].head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Loan_Tenure_Submitted_Missing, dtype: float64

### 11. LoggedIn, Salary_Account dropped

In [53]:
train.drop(['LoggedIn'], axis=1, inplace = True)

In [54]:
train.drop(['Salary_Account'], axis=1, inplace=True)

In [55]:
train.columns

Index(['Gender', 'Monthly_Income', 'Loan_Amount_Applied',
       'Loan_Tenure_Applied', 'Existing_EMI', 'Mobile_Verified', 'Var5',
       'Var1', 'Processing_Fee', 'Filled_Form', 'Device_Type', 'Var2',
       'Source', 'Var4', 'Disbursed', 'age', 'EMI_Loan_Submitted_Missing',
       'Interest_Rate_Missing', 'Loan_Amount_Submitted_Missing',
       'Loan_Tenure_Submitted_Missing'],
      dtype='object')

### 12. Processing_Fee_Missing created which is 1 if Processing_Fee was missing else 0 | Original variable Processing_Fee dropped

In [56]:
train['Processing_Fee'].isna().sum()

59600

In [57]:
train.loc[train.Processing_Fee.notna(), 'Processing_Fee_Missing'] = 0
train.loc[train.Processing_Fee.isna(), 'Processing_Fee_Missing'] = 1
train.drop(['Processing_Fee'], axis=1, inplace = True)

In [58]:
train['Processing_Fee_Missing'].isna().sum()

0

### 13. Source – top 2 kept as is and all others combined into different category

In [59]:
train['Source'].describe()

count     87020
unique       30
top        S122
freq      38567
Name: Source, dtype: object

In [60]:
train['Source'].value_counts().head()

S122    38567
S133    29885
S159     5599
S143     4332
S127     1931
Name: Source, dtype: int64

In [61]:
#train['Source'].isin(['S122', 'S133']).sum()
#train['Source'][~train['Source'].isin(['S122', 'S133'])].shape
train['Source'][~train['Source'].isin(['S122', 'S133'])] = 'other'


/home/suyog/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
train['Source'].value_counts()

S122     38567
S133     29885
other    18568
Name: Source, dtype: int64

### 14. Numerical and One-Hot-Coding performed

In [63]:
train.dtypes

Gender                            object
Monthly_Income                     int64
Loan_Amount_Applied              float64
Loan_Tenure_Applied              float64
Existing_EMI                     float64
Mobile_Verified                   object
Var5                               int64
Var1                              object
Filled_Form                       object
Device_Type                       object
Var2                              object
Source                            object
Var4                               int64
Disbursed                          int64
age                              float64
EMI_Loan_Submitted_Missing       float64
Interest_Rate_Missing            float64
Loan_Amount_Submitted_Missing    float64
Loan_Tenure_Submitted_Missing    float64
Processing_Fee_Missing           float64
dtype: object

In [64]:
# Import libraries and download example data
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [65]:
train.select_dtypes(include=['float64','int64']).columns
#train.select_dtypes(exclude=['object']).head()

Index(['Monthly_Income', 'Loan_Amount_Applied', 'Loan_Tenure_Applied',
       'Existing_EMI', 'Var5', 'Var4', 'Disbursed', 'age',
       'EMI_Loan_Submitted_Missing', 'Interest_Rate_Missing',
       'Loan_Amount_Submitted_Missing', 'Loan_Tenure_Submitted_Missing',
       'Processing_Fee_Missing'],
      dtype='object')

In [66]:
train.select_dtypes(include=['object']).columns

Index(['Gender', 'Mobile_Verified', 'Var1', 'Filled_Form', 'Device_Type',
       'Var2', 'Source'],
      dtype='object')

In [67]:
#limited levels, so ohe won't generate large no. of feature

In [68]:
train.select_dtypes(include=['object']).describe()

,Gender,Mobile_Verified,Var1,Filled_Form,Device_Type,Var2,Source
count,87020,87020,87020,87020,87020,87020,87020
unique,2,2,19,2,2,7,3
top,Male,Y,HBXX,N,Web-browser,B,S122
freq,49848,56481,59294,67530,64316,37280,38567


In [69]:
#### Define which columns should be encoded vs scaled
columns_to_scale  = train.select_dtypes(include=['float64','int64'])
columns_to_encode = train.select_dtypes(include=['object'])

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe    = OneHotEncoder(sparse=False)

In [70]:
type(columns_to_scale)

pandas.core.frame.DataFrame

In [71]:
type(train['Gender'])

pandas.core.series.Series

In [72]:
# scaling
#scaled_data = pd.DataFrame(scaler.fit_transform(train[['Monthly_Income', 'Loan_Amount_Applied']]))
numeric_features = train.loc[:,['Monthly_Income', 'Loan_Amount_Applied', 'Loan_Tenure_Applied', 'Existing_EMI', 'Var5', 
    'Var4', 'age', 'EMI_Loan_Submitted_Missing', 'Interest_Rate_Missing', 'Loan_Amount_Submitted_Missing', 
    'Loan_Tenure_Submitted_Missing', 'Processing_Fee_Missing']]

numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

numeric_features_standardized.head()

,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Var5,Var4,age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,Loan_Amount_Submitted_Missing,Loan_Tenure_Submitted_Missing,Processing_Fee_Missing
0,-0.017841,0.197287,-0.027571,-0.092808,-0.874985,-1.148484,0.859513,0.683810,0.683810,1.230474,1.230474,0.678279
1,-0.010953,-0.085134,-0.028622,-0.092808,1.417628,0.029566,-0.126473,-1.462377,-1.462377,-0.812685,-0.812685,0.678279
2,-0.016693,1.044549,-0.027921,-0.092808,-0.874985,-1.148484,0.436947,0.683810,0.683810,-0.812685,-0.812685,0.678279
3,-0.010953,2.174231,-0.027571,-0.092808,0.888563,0.029566,-0.408184,0.683810,0.683810,-0.812685,-0.812685,0.678279
4,0.018898,0.762128,-0.028622,0.535426,2.123048,0.029566,0.014382,0.683810,0.683810,-0.812685,-0.812685,0.678279


In [73]:
# ### One Hot Encoding with Pandas   
# train['Gender'] = pd.Categorical(train['Gender'])
# dfDummies = pd.get_dummies(train['Gender'], prefix = 'category')
# df = pd.concat([train, dfDummies], axis=1)
# df.head()

In [74]:
### mass One Hot Encoding with Pandas
ohe_encoded_data = pd.get_dummies(columns_to_encode)
type(ohe_encoded_data)

pandas.core.frame.DataFrame

In [75]:
ohe_encoded_data.sample(5)

,Gender_Female,Gender_Male,Mobile_Verified_N,Mobile_Verified_Y,Var1_HAVC,Var1_HAXA,Var1_HAXB,Var1_HAXC,Var1_HAXF,Var1_HAXM,...,Var2_A,Var2_B,Var2_C,Var2_D,Var2_E,Var2_F,Var2_G,Source_S122,Source_S133,Source_other
74935,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6856,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
30163,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
31746,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
30364,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [76]:
numeric_features_standardized.columns

Index(['Monthly_Income', 'Loan_Amount_Applied', 'Loan_Tenure_Applied',
       'Existing_EMI', 'Var5', 'Var4', 'age', 'EMI_Loan_Submitted_Missing',
       'Interest_Rate_Missing', 'Loan_Amount_Submitted_Missing',
       'Loan_Tenure_Submitted_Missing', 'Processing_Fee_Missing'],
      dtype='object')

In [77]:
train = pd.concat([ohe_encoded_data, numeric_features_standardized, train.loc[:,['Disbursed']]], axis=1)
train.head()

,Gender_Female,Gender_Male,Mobile_Verified_N,Mobile_Verified_Y,Var1_HAVC,Var1_HAXA,Var1_HAXB,Var1_HAXC,Var1_HAXF,Var1_HAXM,...,Existing_EMI,Var5,Var4,age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,Loan_Amount_Submitted_Missing,Loan_Tenure_Submitted_Missing,Processing_Fee_Missing,Disbursed
0,1,0,1,0,0,0,0,0,0,0,...,-0.092808,-0.874985,-1.148484,0.859513,0.683810,0.683810,1.230474,1.230474,0.678279,0
1,0,1,0,1,0,0,0,0,0,0,...,-0.092808,1.417628,0.029566,-0.126473,-1.462377,-1.462377,-0.812685,-0.812685,0.678279,0
2,0,1,0,1,0,0,0,0,0,0,...,-0.092808,-0.874985,-1.148484,0.436947,0.683810,0.683810,-0.812685,-0.812685,0.678279,0
3,0,1,0,1,0,0,0,0,0,0,...,-0.092808,0.888563,0.029566,-0.408184,0.683810,0.683810,-0.812685,-0.812685,0.678279,0
4,0,1,0,1,0,0,0,0,0,0,...,0.535426,2.123048,0.029566,0.014382,0.683810,0.683810,-0.812685,-0.812685,0.678279,0


In [78]:
train[target] = train[target].astype('category')
train[target].describe()

count     87020
unique        2
top           0
freq      85747
Name: Disbursed, dtype: int64

In [79]:
# mass encoding with scikit learn

In [83]:
#train.to_csv("train_modified.csv", encoding='utf-8', index=False)

###  lets define a function which will help us create GBM models and perform cross-validation.

In [ ]:
def modelfit(alg, dtrain, predictors, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain['Disbursed'], cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
    
    if performCV:
        print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

## creating a baseline model 
In this case, the evaluation metric is AUC so using any constant value will give 0.5 as result. Typically, a good baseline can be a GBM model with default parameters, i.e. without any tuning

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, train, predictors)

So, the mean CV score is 0.8319 and we should expect our model to do better than this.

## Parameter Tuning

min_samples_split = 500  
min_samples_leaf = 50  
max_depth = 8  
max_features = ‘sqrt’  
subsample = 0.8    
Please note that all the above are just initial estimates and will be tuned later. Lets take the default learning rate of 0.1 here and check the optimum number of trees for that. For this purpose, we can do a grid search and test out values from 20 to 80 in steps of 10.

#### n_estimators

In [ ]:
# Use range. 

# In Python 2.x it returns a list so all you need is,
# >>> range(11, 17)
# [11, 12, 13, 14, 15, 16]

# In Python 3.x it's a iterator so you need to convert it to a list,
# >>> list(range(11, 17))
# [11, 12, 13, 14, 15, 16]

# Note: Second number is exclusive so here it needs to be 16+1 = 17

# EDIT:

# To response to the question about incrementing by 0.5 the easiest option would probably be to use numpy's arange,

# >>> numpy.arange(11, 17, 0.5)
# array([ 11. ,  11.5,  12. ,  12.5,  13. ,  13.5,  14. ,  14.5,  15. ,
#         15.5,  16. ,  16.5])

In [ ]:
#Choose all predictors except target & IDcols 'min_samples_split':[2,4,6,8,10,20,40,60,100]
predictors = [x for x in train.columns if x not in [target, IDcol]]
param_test1 = {'n_estimators':list(range(20,81,10))}
#param_test1 = {'n_estimators': [20,30,40,50,60,70,80]}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

### Tuning tree-specific parameters

#### max_depth and min_samples_split

In [ ]:
param_test2 = {'max_depth':list(range(5,16,2)), 'min_samples_split':list(range(200,1001,200))}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

Note that, 1000 is an extreme value which we tested. There is a fare chance that the optimum value lies above that. So we should check for some higher values as well

#### min_samples_split and min_samples_leaf

In [ ]:
param_test3 = {'min_samples_split':list(range(1000,2100,200)), 'min_samples_leaf':list(range(30,71,10))}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

Here we get the optimum values as 1400 for min_samples_split and 60 for min_samples_leaf. Also, we can see the CV score increasing to 0.8396 now. Let’s fit the model again on this and have a look at the feature importance.

In [ ]:
modelfit(gsearch3.best_estimator_, train, predictors)

#### max_features

In [ ]:
param_test4 = {'max_features':list(range(7,20,2))}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9, min_samples_split=1400, min_samples_leaf=60, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

### Tuning subsample and making models with lower learning rate

### Finally, we have all the parameters needed. Now, we need to lower the learning rate and increase the number of estimators proportionally

#### Lets decrease the learning rate to half, i.e. 0.05 with twice (120) the number of trees.

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm_tuned_1 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=120,max_depth=9, min_samples_split=1400,min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7)
modelfit(gbm_tuned_1, train, predictors)

#### Now lets reduce to one-tenth of the original value, i.e. 0.01 for 600 trees.

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm_tuned_2 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=600,max_depth=9, min_samples_split=1400,min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7)
modelfit(gbm_tuned_2, train, predictors)

#### Lets decrease to one-twentieth of the original value, i.e. 0.005 for 1200 trees.

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm_tuned_3 = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1400,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True)
modelfit(gbm_tuned_3, train, predictors, performCV=False)

#### Here we see that the score reduced very slightly. So lets run for 1500 trees.

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm_tuned_4 = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=9, min_samples_split=1400, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True)
modelfit(gbm_tuned_4, train, predictors, performCV=False)

Another hack that can be used here is the ‘warm_start’ parameter of GBM. You can use it to increase the number of estimators in small steps and test different values without having to run from starting always.